Example notebook on extending functionality or modifying SNEWPY models

In [1]:
from snewpy.models.ccsn import Sukhbold_2015
from snewpy.neutrino import Flavor

import numpy as np
import astropy.units as u

In [2]:
def double_luminosity(lum):
    """Example of modifications applied to a member of Sukhbold_2015
    
    Parameters
    ----------
    lum : dict
        snewpy.neutrino.Flavor-keyed dictionary containing model luminosity
        
    Returns
    -------
    new_lum : dict
        snewpy.neutrino.Flavor-keyed dictionary containing modified model luminosity
    """
    new_lum = {}
    for key, val in lum.items():
        new_lum.update({key: val*2})
    return new_lum


def halve_pdf(func):
    """Example of modification applied to a member function of Sukhbold_2015
    
    Parameters
    ----------
    func : Object
        Member function of SNEWS model to report mean energy
        
    Return
    ------
    new_func : Object
        function with modified behavior
    """
    def new_func(*args, **kwargs):
        result = func(*args, **kwargs)
        
        # Add modified behavior here, some care may be needed based on expected input
        # In this case, the target is get_initial_spectrum, which returns a dictionary 
        #     and the modification is to reduce the PDF by a factor of 2.
        for key, val in result.items():
            result[key] = val / 2
        return result
    
    return new_func


def ModelOscExtension(model):
    """Provide an extended SNEWPY Model class
    
    Parameters
    ----------
    model : snewpy.models.SupernovaModel
        SNEWPY Model to which the extension is applied
    """
    class ExtendedModel(model):            
            
        def __new__(cls, *args, **kwargs):
            instance = model.__new__(cls, *args, **kwargs)
            
            # Add your modifications to members or member functions here
            
            # Example: Double the luminosity of this model
            instance.luminosity = double_luminosity(instance.luminosity)
            # Example: Halve the PDF of this model
            instance.get_initial_spectrum = halve_pdf(instance.get_initial_spectra)
            return instance
            
    return ExtendedModel

In [3]:
# Initialize
param = Sukhbold_2015.get_param_combinations()[0]
Ext_Sukhbold_2015 = ModelOscExtension(Sukhbold_2015)

ext_model = Ext_Sukhbold_2015(**param)
normal_model = Sukhbold_2015(**param)

In [4]:
# Simple Unit Test for member modification

ext_lum = ext_model.luminosity[Flavor.NU_E_BAR]
normal_lum = normal_model.luminosity[Flavor.NU_E_BAR]

ratio = np.divide(ext_lum, normal_lum, where = np.abs(normal_lum.value) > 1e-10, out=np.zeros_like(ext_lum))
# AstroPy Quantities are prone to floating point errors, 
#    so I am using a tolerance of 1e-10 here
np.all(ratio[ratio!=0] == 2)

True

In [5]:
# Simple Unit Test for member function modification

t = 0.5 * u.s
E = np.arange(10) * u.MeV

ext_pdf = ext_model.get_initial_spectra(t, E)[Flavor.NU_E_BAR].value
normal_pdf = normal_model.get_initial_spectra(t, E)[Flavor.NU_E_BAR].value

ratio = np.divide(ext_pdf, normal_pdf, where = np.abs(normal_pdf) > 1e-10, out=np.zeros_like(ext_pdf))
# AstroPy Quantities are prone to floating point errors, 
#    so I am using a tolerance of 1e-10 here
np.all((ratio - 2) < 1e-10)

True